## Installing and Importing the necessary libraries

In [146]:
!pip3 install pandas-gbq

In [36]:
import pandas as pd
import json
from google.cloud import storage
from pyspark.sql import SparkSession
import re
import ast
from google.cloud import bigquery
import numpy as np
import hashlib

In [43]:
def ingest_customer_data(INPUT_PATH, OUTPUT_PATH):
    '''
    Function to ingest the Customer Data
    '''
    data = pd.read_json(INPUT_PATH, lines=True)
    df = pd.DataFrame()
    customer_name1 = []
    customer_id1 = []
    Gender = []
    zip_code = []
    connection_type = []
    connection_plan = []
    new_ported = []
    old_operator = []
    curr_operator = []
    phone_number = []
    contract_date = []
    SSN = []
    DL = []
    for lab, row in data.iterrows():
        customer_name1.append(row['root']['cus_name'])
        customer_id1.append(row['root']['cusomer_id'])
        Gender.append(row['root']['gender'])
        connection_type.append(row['connection']['type'])
        connection_plan.append(row['connection']['plan'])
        zip_code.append(row['identities']['DL']['address']['zip_code'])
        new_ported.append(row['phone']['new/port']['ported'])
        old_operator.append(row['phone']['new/port']['old_operator'])
        phone_number.append(row['phone']['number'])
        curr_operator.append(row['phone']['operator'])
        contract_date.append(row['phone']['start_date'])
        SSN.append(row['identities']['SSN']['number'])
        DL.append(row['identities']['DL']['number'])
    df['customer_id'] = customer_id1
    df['customer_name'] = customer_name1
    df['phone_number'] = phone_number
    df['zip_code'] = zip_code
    df['Gender'] = Gender
    df['connection_type'] = connection_type
    df['connection_plan'] = connection_plan
    df['new_ported'] = new_ported
    df['old_operator'] = old_operator
    df['curr_operator'] = curr_operator
    df['contract_date'] = contract_date
    df['SSN'] = SSN
    df['DL'] = DL
    df['SSN_hashed'] = df['SSN'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
    df['DL_hashed'] = df['DL'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
    df.drop(['SSN', 'DL'], axis = 1, inplace = True)
    df.to_gbq(OUTPUT_PATH,  project_id='acn-in-cf-data-ggl-aca-c01-t04', if_exists='replace')

## Reading Customer Data

In [44]:
ingest_customer_data("gs://in-telecom-t4/Test/customer_data.json", 'test1.customer_data_id1')

1it [00:13, 13.19s/it]


## EDA of a sample of the customer data

In [5]:
data = pd.read_json("gs://in-telecom-t4/Test/customer_data.json", lines=True)

In [25]:
for cols in data.columns:
    print(data[cols][0])

{'cus_name': 'Heather Johnson', 'cusomer_id': '028146e6-edc8-4b4e-8d87-69fc952d4f01', 'marital_status': 'Married', 'gender': 'Female', 'DOB': '19840105', 'occupation': 'Information Technology', 'address': {'cus_home': 'Unit 5824 Box 8279', 'cus_city': 'DPO AE 03217', 'cus_zipcode': '03217', 'cus_state': 'AE'}}
{'SSN': {'number': '106-12-5365'}, 'DL': {'number': 'UY1814444734234', 'address': {'home': 'Unit 5824 Box 8279', 'city': 'DPO AE 03217', 'state': 'AE', 'zip_code': '03217'}}}
{'name': 'Jamie Scott', 'address': {'home': '51348 Baker Glens', 'city': 'Lake Deborahchester', 'state': 'TN', 'zipcode': '61108'}}
{'plan': 'Both', 'type': 'Prepaid'}
{'number': '5034147053569', 'operator': 'P-Mobile', 'start_date': '20141122', 'new/port': {'ported': True, 'old_operator': 'Martinez LLC'}}
{'number': 2034378496853, 'operator': 'Bird PLC'}


In [7]:
data[['root']].loc[0]

root    {'cus_name': 'Heather Johnson', 'cusomer_id': ...
Name: 0, dtype: object

In [8]:
storage_client = storage.Client()
bucket = storage_client.get_bucket("in-telecom-t4")
blob = bucket.get_blob("Test/customer_data.json")
multi_json = blob.download_as_text()

In [10]:
multi_json[:100]

'{"root":{"cus_name":"Heather Johnson","cusomer_id":"028146e6-edc8-4b4e-8d87-69fc952d4f01","marital_s'

In [11]:
json_file = multi_json.replace('new\\/port', 'new_port')

In [12]:
json_file[:100]

'{"root":{"cus_name":"Heather Johnson","cusomer_id":"028146e6-edc8-4b4e-8d87-69fc952d4f01","marital_s'

In [13]:
file = json.dumps(json_file)

In [14]:
data.head()

,root,identities,seller,connection,phone,other_phone
0,"{'cus_name': 'Heather Johnson', 'cusomer_id': ...","{'SSN': {'number': '106-12-5365'}, 'DL': {'num...","{'name': 'Jamie Scott', 'address': {'home': '5...","{'plan': 'Both', 'type': 'Prepaid'}","{'number': '5034147053569', 'operator': 'P-Mob...","{'number': 2034378496853, 'operator': 'Bird PLC'}"
1,"{'cus_name': 'Dustin Romero', 'cusomer_id': 'd...","{'SSN': {'number': '858-81-1906'}, 'DL': {'num...","{'name': 'Kaitlyn Knight', 'address': {'home':...","{'plan': 'Data only', 'type': 'Prepaid'}","{'number': 7613062600667, 'operator': 'P-Mobil...","{'number': 5614253555384, 'operator': 'Bird PLC'}"
2,"{'cus_name': 'Keith Rodriguez', 'cusomer_id': ...","{'SSN': {'number': '680-85-8589'}, 'DL': {'num...","{'name': 'Alexandra Blackwell', 'address': {'h...","{'plan': 'Both', 'type': 'Prepaid'}","{'number': '3978831593479', 'operator': 'P-Mob...","{'number': 4353033341134, 'operator': 'Cole-Mo..."
3,"{'cus_name': 'Ashley Watson', 'cusomer_id': '2...","{'SSN': {'number': '931-32-1199'}, 'DL': {'num...","{'name': 'Michael Smith', 'address': {'home': ...","{'plan': 'Both', 'type': 'Post-paid'}","{'number': '2359928553481', 'operator': 'P-Mob...","{'number': 8696190152988, 'operator': 'Bird PLC'}"
4,"{'cus_name': 'John Simmons', 'cusomer_id': 'a8...","{'SSN': {'number': '191-23-9845'}, 'DL': {'num...","{'name': 'Brianna Jarvis', 'address': {'home':...","{'plan': 'Both', 'type': 'Prepaid'}","{'number': '5427481149458', 'operator': 'P-Mob...","{'number': 1795999414788, 'operator': 'Cole-Mo..."


In [29]:
df = pd.DataFrame()
customer_name1 = []
customer_id1 = []
Gender = []
zip_code = []
connection_type = []
connection_plan = []
new_ported = []
old_operator = []
curr_operator = []
phone_number = []
contract_date = []
SSN = []
DL = []

In [30]:
for lab, row in data.iterrows():
    customer_name1.append(row['root']['cus_name'])
    customer_id1.append(row['root']['cusomer_id'])
    Gender.append(row['root']['gender'])
    connection_type.append(row['connection']['type'])
    connection_plan.append(row['connection']['plan'])
    zip_code.append(row['identities']['DL']['address']['zip_code'])
    new_ported.append(row['phone']['new/port']['ported'])
    old_operator.append(row['phone']['new/port']['old_operator'])
    phone_number.append(row['phone']['number'])
    curr_operator.append(row['phone']['operator'])
    contract_date.append(row['phone']['start_date'])
    SSN.append(row['identities']['SSN']['number'])
    DL.append(row['identities']['DL']['number'])

In [31]:
df['customer_id'] = customer_id1
df['customer_name'] = customer_name1
df['phone_number'] = phone_number
df['zip_code'] = zip_code
df['Gender'] = Gender
df['connection_type'] = connection_type
df['connection_plan'] = connection_plan
df['new_ported'] = new_ported
df['old_operator'] = old_operator
df['curr_operator'] = curr_operator
df['contract_date'] = contract_date
df['SSN'] = SSN
df['DL'] = DL
df.replace('nan', np.nan, inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customer_id      45000 non-null  object
 1   customer_name    45000 non-null  object
 2   phone_number     45000 non-null  object
 3   zip_code         45000 non-null  object
 4   Gender           45000 non-null  object
 5   connection_type  45000 non-null  object
 6   connection_plan  45000 non-null  object
 7   new_ported       45000 non-null  bool  
 8   old_operator     8641 non-null   object
 9   curr_operator    45000 non-null  object
 10  contract_date    45000 non-null  object
 11  SSN              45000 non-null  object
 12  DL               45000 non-null  object
dtypes: bool(1), object(12)
memory usage: 4.2+ MB


In [35]:
df.head()
#df.info()

,customer_id,customer_name,phone_number,zip_code,Gender,connection_type,connection_plan,new_ported,old_operator,curr_operator,contract_date,SSN,DL
0,028146e6-edc8-4b4e-8d87-69fc952d4f01,Heather Johnson,5034147053569,03217,Female,Prepaid,Both,True,Martinez LLC,P-Mobile,20141122,106-12-5365,UY1814444734234
1,df6906df-9746-4185-9484-907e27c5d3aa,Dustin Romero,7613062600667,93738,Male,Prepaid,Data only,False,None,P-Mobile,20100428,858-81-1906,ZF2203268944814
2,625aa620-616e-4dd1-8858-cf3aaad0b177,Keith Rodriguez,3978831593479,10640,Male,Prepaid,Both,False,None,P-Mobile,20120623,680-85-8589,CK7472158497751
3,20d5cd7f-f608-4405-be4b-1d89926140b3,Ashley Watson,2359928553481,49907,Female,Post-paid,Both,False,None,P-Mobile,20110112,931-32-1199,OG3259936936767
4,a8118489-9d3b-4fcf-a538-985b4281d557,John Simmons,5427481149458,45010,Female,Prepaid,Both,False,None,P-Mobile,20101211,191-23-9845,QC7384771847743


## Removing PII using Hashing

In [38]:
df['SSN_hashed'] = df['SSN'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

In [39]:
df['DL_hashed'] = df['DL'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

In [41]:
df.drop(['SSN', 'DL'], axis = 1, inplace = True)

In [42]:
df.head()

,customer_id,customer_name,phone_number,zip_code,Gender,connection_type,connection_plan,new_ported,old_operator,curr_operator,contract_date,SSN_hashed,DL_hashed
0,028146e6-edc8-4b4e-8d87-69fc952d4f01,Heather Johnson,5034147053569,03217,Female,Prepaid,Both,True,Martinez LLC,P-Mobile,20141122,4972586b084d7e8b8b8c68d0de8ff502e8506f6e6d1ba3...,63a1032f3b6fdcdeba0bab124c87935145305a40cd9c1e...
1,df6906df-9746-4185-9484-907e27c5d3aa,Dustin Romero,7613062600667,93738,Male,Prepaid,Data only,False,None,P-Mobile,20100428,ea5578e23543aa8ad343137af02b061a6819679b9a4f3c...,f9ed78fe9efd9617b478128bc5538a4ee76b6b539c3b0d...
2,625aa620-616e-4dd1-8858-cf3aaad0b177,Keith Rodriguez,3978831593479,10640,Male,Prepaid,Both,False,None,P-Mobile,20120623,040d31cd1e7a03787912a2dec29516fa02c8293e230937...,3a33724ad999c62d7099bd990373c019c1c6cc7d10c164...
3,20d5cd7f-f608-4405-be4b-1d89926140b3,Ashley Watson,2359928553481,49907,Female,Post-paid,Both,False,None,P-Mobile,20110112,7a21b024ece083e2159defe8244e9754d0233b16f246c6...,e54b320bcee43ef7ed591c0c72e19bc268011af80120d6...
4,a8118489-9d3b-4fcf-a538-985b4281d557,John Simmons,5427481149458,45010,Female,Prepaid,Both,False,None,P-Mobile,20101211,3a888ce0bb95e46fa443aaf48c9142dbf92446f483be69...,e68abeffce8b6183ad24f8e6249c5f86214764a530ab6f...


## Loading Data to BigQuery

In [147]:
df.to_gbq('test1.test_customer_data',  project_id='acn-in-cf-data-ggl-aca-c01-t04', if_exists='replace')

1it [00:07,  7.27s/it]


In [149]:
df['old_operator'].value_counts()

Gardner Inc     2203
Cole-Moreno     2196
Bird PLC        2157
Martinez LLC    2085
Name: old_operator, dtype: int64